<a href="https://colab.research.google.com/github/dkimds/goormthon/blob/main/naver_news_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 뉴스 크롤러 만들기 연습
아래 코드를 실행해야 이하부분이 돌아갑니다.

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import time
import re

## 네이버 뉴스 간추리기



- 뉴스 주제: 챗봇
- 기간: 1개월

In [ ]:
url = "https://search.naver.com/search.naver?where=news&query=gpt&sm=tab_opt&sort=0&photo=0&field=0&pd=2&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3A1m&is_sug_officeid=0&office_category=0&service_area=0"

In [ ]:
web = requests.get(url).content
source = BeautifulSoup(web, 'html.parser')

In [ ]:
news_subjects = source.find_all('a', {'class' : 'news_tit'}) # ResultSet (리스트와 유사한 형태)

subject_list = []

for subject in news_subjects:
    subject_list.append(subject.get_text())

print(subject_list)

[]


In [ ]:
urls = news_subjects
first_article = urls[0]
first_article.attrs # tag's attributes (== attrs)

{'href': 'https://www.yna.co.kr/view/AKR20231130006500091?input=1195m',
 'class': ['news_tit'],
 'target': '_blank',
 'onclick': "return goOtherCR(this, 'a=nws*h.tit&r=1&i=880000D8_000000000000000014362313&g=001.0014362313&u='+urlencode(this.href));",
 'title': '[챗GPT 1년] 인터넷·아이폰 넘은 AI혁명…부작용 우려에 규제 필요성 대두'}

In [ ]:
first_article.attrs['href'] # tag의 attribute 'href'의 value 꺼내기

'https://www.yna.co.kr/view/AKR20231130006500091?input=1195m'

In [ ]:
for urls in source.find_all('a', {'class' : "news_tit"}):
    print(urls.attrs['href'])

https://www.yna.co.kr/view/AKR20231130006500091?input=1195m
https://www.hankookilbo.com/News/Read/A2023112318540001009?did=NA
http://www.newsis.com/view/?id=NISX20231128_0002538210&cID=10201&pID=10200
http://www.segye.com/newsView/20231129515385?OutUrl=naver
http://www.yonhapnewstv.co.kr/MYH20231130010600641/?input=1825m
http://www.itdaily.kr/news/articleView.html?idxno=218497
https://www.etnews.com/20231130000256
https://www.hankookilbo.com/News/Read/A2023112904350005517?did=NA
https://www.hankookilbo.com/News/Read/A2023112722110004787?did=NA
https://www.mk.co.kr/article/10886457


In [ ]:
sample_url = 'https://n.news.naver.com/mnews/article/003/0011379817?sid=102'
sample_url.startswith('https://n.news.naver.com/')

True

In [ ]:
urls_list = []

for urls in source.find_all('a', {'class' : "info"}):
    if urls.attrs["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls.attrs["href"])

urls_list

['https://n.news.naver.com/mnews/article/001/0014362313?sid=104',
 'https://n.news.naver.com/mnews/article/469/0000772918?sid=104',
 'https://n.news.naver.com/mnews/article/003/0012236267?sid=102',
 'https://n.news.naver.com/mnews/article/022/0003880120?sid=104',
 'https://n.news.naver.com/mnews/article/422/0000632048?sid=101',
 'https://n.news.naver.com/mnews/article/030/0003161687?sid=105',
 'https://n.news.naver.com/mnews/article/469/0000772905?sid=104',
 'https://n.news.naver.com/mnews/article/469/0000772895?sid=104',
 'https://n.news.naver.com/mnews/article/009/0005222445?sid=004']

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

web_news = requests.get(urls_list[0], headers=headers).content
source_news = BeautifulSoup(web_news, 'html.parser')

## 단일 뉴스 페이지 분석

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}

web_news = requests.get(urls_list[0], headers=headers).content
source_news = BeautifulSoup(web_news, 'html.parser')

1) 기사제목과 발행일자

In [ ]:
title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
print(title)

[챗GPT 1년] 인터넷·아이폰 넘은 AI혁명…부작용 우려에 규제 필요성 대두


In [ ]:
date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()
date = date.replace(" ","")
date1 = date[:11] # 날짜까지 파싱
date2 = date[13:] # 시간부분 파싱
date3 = (lambda x : 'am' if x == '오전' else 'pm')(date[11:13])
date4 = date1 + date2 + date3
date = pd.Timestamp(date4)
print(date)

2023-11-30 05:53:00


2) 기사본문

In [ ]:
article = source_news.find('article', {'id' : 'dic_area'}).get_text()
article

'\n예상 못한 열풍…애초 이름은 \'챗위드GPT\', 출시 5분 전에 \'위드\' 빠져숙제·여행계획 등 일상질문 답변부터 문학·예술 창작, 판결문 작성까지업계 지각변동 속 기술개발 무한경쟁…가짜뉴스 확산·범죄악용 차단 시급\n\n\n\n챗GPT 로고[로이터 연합뉴스 자료사진. 재판매 및 DB 금지](샌프란시스코=연합뉴스) 김태종 특파원 = 사람처럼 묻고 답하는 생성형 인공지능(AI) 챗GPT가 30일로 \'첫 돌\'을 맞았다.    세상에 나온 지 채 1년 만에 챗GPT는 전 세계 생성형 AI 열풍을 일으켰고, 개발사 오픈AI는 일약 세계 최고 기업 중 하나로 도약했다.    챗GPT의 1년은 역대 가장 임팩트 있는 기술로 받아들여지고 있다.     1990년대 인터넷, 2000년대 아이폰을 뛰어넘는 혁명을 만들어내고 있다는 평가가 나온다.    챗GPT를 계기로 AI 기술은 이미 생활 속에 깊숙이 자리를 잡아가고 있지만, 악용의 우려도 커지면서 규제 필요성도 제기되고 있다. \n\n\n\n오픈AI와 챗GPT 로고[AFP 연합뉴스 자료사진. 재판매 및 DB 금지] 출시 두 달만에 이용자 1억명…기업가치 111조원, CEO 거취 \'진통\'     이제는 익숙해진 챗GPT라는 이름은 애초 \'챗위드GPT\'(Chat With GPT)였다. 오픈AI의 대규모 언어 모델(LLM)인 \'GPT와 대화한다\'는 의미로 지어졌다.    그러나 이는 세상에 존재를 알리기 직전 현재의 챗GPT로 바뀌었다.    챗GPT를 디자인하는 데 관여한 오픈AI 한 관계자는 "출시 예정 시간 바로 5분 전 \'챗위드GPT\'에서 지금의 챗GPT로 바뀌었다"고 밝혔다.    이름이 갑작스럽게 바뀐 이유는 간단했다. 이 관계자는 "이름이 너무 거추장스럽게 길다는 지적이 나왔고, 그래서 그냥 \'위드\'(With)를 뺐다"고 설명했다.    어렵지 않게 이름을 바꿀 수 있었던 데에는 챗GPT가 지금과 같은 열풍을 일으키리라고는 오픈AI도 전혀 예상하지 못했기 때문이다.     이 관계자는

In [ ]:
article = article.replace("\n", "")
article = article.replace("\'", "")
article = article.strip()
article

'예상 못한 열풍…애초 이름은 챗위드GPT, 출시 5분 전에 위드 빠져숙제·여행계획 등 일상질문 답변부터 문학·예술 창작, 판결문 작성까지업계 지각변동 속 기술개발 무한경쟁…가짜뉴스 확산·범죄악용 차단 시급챗GPT 로고[로이터 연합뉴스 자료사진. 재판매 및 DB 금지](샌프란시스코=연합뉴스) 김태종 특파원 = 사람처럼 묻고 답하는 생성형 인공지능(AI) 챗GPT가 30일로 첫 돌을 맞았다.    세상에 나온 지 채 1년 만에 챗GPT는 전 세계 생성형 AI 열풍을 일으켰고, 개발사 오픈AI는 일약 세계 최고 기업 중 하나로 도약했다.    챗GPT의 1년은 역대 가장 임팩트 있는 기술로 받아들여지고 있다.     1990년대 인터넷, 2000년대 아이폰을 뛰어넘는 혁명을 만들어내고 있다는 평가가 나온다.    챗GPT를 계기로 AI 기술은 이미 생활 속에 깊숙이 자리를 잡아가고 있지만, 악용의 우려도 커지면서 규제 필요성도 제기되고 있다. 오픈AI와 챗GPT 로고[AFP 연합뉴스 자료사진. 재판매 및 DB 금지] 출시 두 달만에 이용자 1억명…기업가치 111조원, CEO 거취 진통     이제는 익숙해진 챗GPT라는 이름은 애초 챗위드GPT(Chat With GPT)였다. 오픈AI의 대규모 언어 모델(LLM)인 GPT와 대화한다는 의미로 지어졌다.    그러나 이는 세상에 존재를 알리기 직전 현재의 챗GPT로 바뀌었다.    챗GPT를 디자인하는 데 관여한 오픈AI 한 관계자는 "출시 예정 시간 바로 5분 전 챗위드GPT에서 지금의 챗GPT로 바뀌었다"고 밝혔다.    이름이 갑작스럽게 바뀐 이유는 간단했다. 이 관계자는 "이름이 너무 거추장스럽게 길다는 지적이 나왔고, 그래서 그냥 위드(With)를 뺐다"고 설명했다.    어렵지 않게 이름을 바꿀 수 있었던 데에는 챗GPT가 지금과 같은 열풍을 일으키리라고는 오픈AI도 전혀 예상하지 못했기 때문이다.     이 관계자는 "사실 하루에 20만∼30만명 정도가 들어오겠지, 생각했다"며 "단기간에 수천만 명이 이

불필요한 기호와 이메일주소를 삭제

In [ ]:
import re
news_contents = article
# 기자 이메일 주소부터 그 이후 전체 삭제하기
pattern = re.compile(r'[\s\Wa-zA-Z0-9]*@')
email_address = pattern.search(news_contents)
news_contents = news_contents[:email_address.start()]

print(news_contents)

예상 못한 열풍…애초 이름은 챗위드GPT, 출시 5분 전에 위드 빠져숙제·여행계획 등 일상질문 답변부터 문학·예술 창작, 판결문 작성까지업계 지각변동 속 기술개발 무한경쟁…가짜뉴스 확산·범죄악용 차단 시급챗GPT 로고[로이터 연합뉴스 자료사진. 재판매 및 DB 금지](샌프란시스코=연합뉴스) 김태종 특파원 = 사람처럼 묻고 답하는 생성형 인공지능(AI) 챗GPT가 30일로 첫 돌을 맞았다.    세상에 나온 지 채 1년 만에 챗GPT는 전 세계 생성형 AI 열풍을 일으켰고, 개발사 오픈AI는 일약 세계 최고 기업 중 하나로 도약했다.    챗GPT의 1년은 역대 가장 임팩트 있는 기술로 받아들여지고 있다.     1990년대 인터넷, 2000년대 아이폰을 뛰어넘는 혁명을 만들어내고 있다는 평가가 나온다.    챗GPT를 계기로 AI 기술은 이미 생활 속에 깊숙이 자리를 잡아가고 있지만, 악용의 우려도 커지면서 규제 필요성도 제기되고 있다. 오픈AI와 챗GPT 로고[AFP 연합뉴스 자료사진. 재판매 및 DB 금지] 출시 두 달만에 이용자 1억명…기업가치 111조원, CEO 거취 진통     이제는 익숙해진 챗GPT라는 이름은 애초 챗위드GPT(Chat With GPT)였다. 오픈AI의 대규모 언어 모델(LLM)인 GPT와 대화한다는 의미로 지어졌다.    그러나 이는 세상에 존재를 알리기 직전 현재의 챗GPT로 바뀌었다.    챗GPT를 디자인하는 데 관여한 오픈AI 한 관계자는 "출시 예정 시간 바로 5분 전 챗위드GPT에서 지금의 챗GPT로 바뀌었다"고 밝혔다.    이름이 갑작스럽게 바뀐 이유는 간단했다. 이 관계자는 "이름이 너무 거추장스럽게 길다는 지적이 나왔고, 그래서 그냥 위드(With)를 뺐다"고 설명했다.    어렵지 않게 이름을 바꿀 수 있었던 데에는 챗GPT가 지금과 같은 열풍을 일으키리라고는 오픈AI도 전혀 예상하지 못했기 때문이다.     이 관계자는 "사실 하루에 20만∼30만명 정도가 들어오겠지, 생각했다"며 "단기간에 수천만 명이 이용

3) 기사 발행사

In [ ]:
press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()
print(press_company)

연합뉴스


### 여러 뉴스 모으기

In [ ]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from datetime import datetime
import time
import re

In [ ]:
url = "https://search.naver.com/search.naver?where=news&query=chatgpt&sm=tab_opt&sort=0&photo=0&field=0&pd=2&ds=&de=&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3A1m&is_sug_officeid=0&office_category=0&service_area=0web = requests.get(url).content"
web = requests.get(url).content

source = BeautifulSoup(web, 'html.parser')

urls_list = []

for urls in source.find_all('a', {'class' : "info"}):
    if urls["href"].startswith("https://n.news.naver.com"):
        urls_list.append(urls["href"])

urls_list

['https://n.news.naver.com/mnews/article/032/0003263907?sid=105',
 'https://n.news.naver.com/mnews/article/009/0005223085?sid=104',
 'https://n.news.naver.com/mnews/article/640/0000046593?sid=110']

In [ ]:
categories = []
titles = []
dates = []
articles = []
article_urls = []
press_companies = []

In [ ]:
for url in urls_list:
    try:
        headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
        web_news = requests.get(url, headers=headers).content
        source_news = BeautifulSoup(web_news, 'html.parser')

        # 1) 카테고리 class="media_end_categorize_item"
        category = source_news.find('em',{'class' : 'media_end_categorize_item'}).get_text()
        # 2) 기사 제목
        title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
        print('Processing article : {}'.format(title))

        # 3) 기사 날짜
        date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

        # 4) 기사 본문
        article = source_news.find('article', {'id' : 'dic_area'}).get_text()
        article = article.replace("\n", "")
        article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
        article = article.replace("동영상 뉴스       ", "")
        article = article.replace("동영상 뉴스", "")
        article = article.strip()

        # 5) 기사 발행 언론사
        press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()

        # 위 2~5를 통해 성공적으로 제목/날짜/본문/언론사 정보가 모두 추출되었을 때에만 리스트에 추가해 길이를 동일하게 유지해줍니다.
        categories.append(category)
        titles.append(title)
        dates.append(date)
        articles.append(article)
        article_urls.append(url) # 6) 기사 URL
        press_companies.append(press_company)

    except:
        print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))


Processing article : 코딩 전혀 몰라도 나만의 챗봇 뚝딱[ChatGPT AI 빅뱅 1년]
Processing article : Multi-modal AI gains global traction as technology advances
Processing article : Imagination is key to prosperity in the AI century


In [ ]:
# 각 데이터 종류별 list에 담아둔 전체 데이터를 DataFrame에 모아둡니다.
article_df = pd.DataFrame({'Category':categories,
                           'Title':titles,
                           'Date':dates,
                           'Article':articles,
                           'URL':article_urls,
                           'PressCompany':press_companies})
article_df

,Category,Title,Date,Article,URL,PressCompany
0,IT,코딩 전혀 몰라도 나만의 챗봇 뚝딱[ChatGPT AI 빅뱅 1년],2023.11.28. 오전 6:03,"(상) 일상 파고든 AI최신 버전 GPT4, 음성도 인식소비자의 ‘AI 접근’ 문턱...",https://n.news.naver.com/mnews/article/032/000...,경향신문
1,세계,Multi-modal AI gains global traction as techno...,2023.11.30. 오후 2:44,[Courtesy of Soul Machines]Artificial intellig...,https://n.news.naver.com/mnews/article/009/000...,매일경제
2,오피니언,Imagination is key to prosperity in the AI cen...,2023.11.29. 오후 8:11,"Yeom Jae-ho The author, a former president of ...",https://n.news.naver.com/mnews/article/640/000...,코리아중앙데일리


# 제출 평가 부분

In [ ]:
def main_crawling(query, start_date, end_date, sort_type, max_page):
    '''
    뉴스를 검색하여 데이터 프레임으로 반환
    input args::
        query: 검색하려는 뉴스주제
        start_date: 뉴스검색이 시작되는 날짜
        end_date: 뉴스검색이 끝나는 날짜
        sort_type: 정렬타입. 관련도순 = 0, 최신순 = 1, 오래된순 = 2
        max_page: 
    return:
        데이터 필드: 날짜, 카테고리, 발행사, 제목, 기사문, 기사주소
    '''
    if query == '':
        query = '데이터 분석'
    if len(start_date) != 10:
        start_date = '2021.01.01'
    if len(end_date) != 10:
        end_date = '2021.12.31'
    if sort_type not in ['0', '1', '2']:
        sort_type = '0'
    if max_page == '':
        max_page = 5


    # 각 기사들의 데이터를 종류별로 나눠담을 리스트를 생성합니다. (추후 DataFrame으로 모을 예정)
    categories = []
    titles = []
    dates = []
    articles = []
    article_urls = []
    press_companies = []

    # 주어진 일자를 쿼리에 맞는 형태로 변경해줍니다.
    start_date = start_date.replace(".", "")
    end_date = end_date.replace(".", "")

    # 지정한 기간 내 원하는 페이지 수만큼의 기사를 크롤링합니다.
    current_call = 1
    last_call = (max_page - 1) * 10 + 1 # max_page이 5일 경우 41에 해당


    while current_call <= last_call:

        print('\n{}번째 기사글부터 크롤링을 시작합니다.'.format(current_call))

        url = "https://search.naver.com/search.naver?where=news&query=" + query \
              + "&sort=" + sort_type \
              + "&nso=so%3Ar%2Cp%3Afrom" + start_date \
              + "to" + end_date \
              + "%2Ca%3A&start=" + str(current_call)

        urls_list = []
        try: # 네이버 뉴스 검색결과 페이지 자체에 접근이 불가능할 경우 에러가 발생할 수 있습니다.
            web = requests.get(url).content
            source = BeautifulSoup(web, 'html.parser')

            for urls in source.find_all('a', {'class' : "info"}):
                if urls["href"].startswith("https://n.news.naver.com"):
                    urls_list.append(urls["href"])
        except:
            print('해당 뉴스 검색 페이지의 네이버 뉴스 링크를 모으는 중 에러가 발생했습니다. : ', url)

        # urls_list : 해당 페이지에 있는 "네이버 뉴스"의 링크 모음(list)
        if urls_list != []:
            for url in urls_list:
                try: # 특정 뉴스 기사글 하나를 크롤링하는 중 에러가 발생할 수 있습니다.ㄴ
                    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
                    web_news = requests.get(url, headers=headers).content
                    source_news = BeautifulSoup(web_news, 'html.parser')
                    # 1) 카테고리 class="media_end_categorize_item"
                    category = source_news.find('em',{'class' : 'media_end_categorize_item'}).get_text()

                    title = source_news.find('h2', {'class' : 'media_end_head_headline'}).get_text()
                    print('Processing article : {}'.format(title))

                    date = source_news.find('span', {'class' : 'media_end_head_info_datestamp_time'}).get_text()

                    article = source_news.find('article', {'id' : 'dic_area'}).get_text()
                    article = article.replace("\n", "")
                    article = article.replace("\'", "")
                    # article = article.replace("// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")
                    # article = article.replace("동영상 뉴스       ", "")
                    # article = article.replace("동영상 뉴스", "")
                    article = article.strip()

                    press_company = source_news.find('em', {'class':'media_end_linked_more_point'}).get_text()

                    categories.append(category)
                    titles.append(title)
                    dates.append(date)
                    articles.append(article)
                    press_companies.append(press_company)
                    article_urls.append(url)
                    time.sleep(1)
                except:
                    print('\n*** {}번부터 {}번까지의 기사글을 크롤링하는 중 문제가 발생했습니다.'.format(current_call, current_call+9))
                    print('*** 다음 링크의 뉴스를 크롤링하는 중 에러가 발생했습니다 : {}'.format(url))

        else:
            pass

        current_call += 10


    article_df = pd.DataFrame({'date':dates,
                               'category':categories,
                               'press':press_companies,
                               'title':titles,
                               'document':articles,
                               'link':article_urls,
                               })

    # article_df.to_csv('result_{}.csv'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

    print('\n크롤링이 성공적으로 완료되었습니다!')
    print('\n크롤링 결과를 다음 파일에 저장하였습니다 : {}'.format(datetime.now().strftime('%y%m%d_%H%M')))

    return article_df

In [ ]:
query = input('검색어를 입력해주세요. (ex. 데이터 분석) : ')
start_date = input('검색 시작 날짜를 입력해주세요. (형식 : 2021.01.01) : ')
end_date = input('검색 종료 날짜를 입력해주세요. (형식 : 2021.12.31) : ')
sort_type = input('정렬 타입을 입력해주세요 (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : ')
max_page = input('크롤링을 원하는 전체 페이지 수를 입력해주세요. (ex. 5) : ')

if start_date > end_date:
    print('\n시작 날짜는 종료 날짜보다 이후로 지정하실 수 없습니다. 다시 실행해주세요!')
elif max_page == '':
    max_page = 5
    print('\n원하시는 페이지 수가 입력되지 않았습니다. 5 페이지까지만 크롤링을 진행합니다.')
    main_crawling(query, start_date, end_date, sort_type, max_page)
else:
    max_page = int(max_page)
    scraped_news = main_crawling(query, start_date, end_date, sort_type, max_page)
    scraped_news.to_csv('result_{}.csv'.format(datetime.now().strftime('%y%m%d_%H%M')), index=False, encoding='utf-8')

검색어를 입력해주세요. (ex. 데이터 분석) : 인공지능
검색 시작 날짜를 입력해주세요. (형식 : 2021.01.01) : 2023.11.28
검색 종료 날짜를 입력해주세요. (형식 : 2021.12.31) : 2023.11.30
정렬 타입을 입력해주세요 (관련도순 = 0, 최신순 = 1, 오래된순 = 2) : 2
크롤링을 원하는 전체 페이지 수를 입력해주세요. (ex. 5) : 5

1번째 기사글부터 크롤링을 시작합니다.
Processing article : [알림] 뉴스1 콘텐츠 저작권 고지
Processing article : 연말 '인사폭풍' 부는 통신3사, 거세지는 '탈통신' 움직임
Processing article : [제59회 중앙광고대상] AI로 연결된 세상, 친근한 메시지로 다가간 듯
Processing article : 정의선, 미국 오토모티브뉴스 ‘자동차 산업 올해의 리더’

11번째 기사글부터 크롤링을 시작합니다.

21번째 기사글부터 크롤링을 시작합니다.

31번째 기사글부터 크롤링을 시작합니다.

41번째 기사글부터 크롤링을 시작합니다.

크롤링이 성공적으로 완료되었습니다!

크롤링 결과를 다음 파일에 저장하였습니다 : 231130_0912
